# Get Metrics of the Graph model on test set graphs

In [ ]:
from func.ultis import load_obj
import networkx as nx
import numpy as np

In [5]:
set_test = load_obj("lrp_training_graphs/graphs_dataset_train_with_augmentations_LRP_retrained_skript_0")
print(len(set_test))

4


In [6]:
for graph in set_test:
    labels_dict = nx.get_node_attributes(graph, "label")
    labels = []
    for label in labels_dict.items():
        #print(label[1])
        labels.append(label[1])
    negatives = np.count_nonzero(labels)
    positives = len(labels) - negatives
    print(f"neg: {negatives}, \t pos: {positives}")


neg: 50397, 	 pos: 4041
neg: 45161, 	 pos: 3362
neg: 2436, 	 pos: 3874
neg: 38748, 	 pos: 2807


In [8]:
from func.graph_models import GCN
import torch

model = GCN(3, num_classes=1)
checkpoint_graph = 'output/graph_model_focal_LRP_retrained.pt'
model.load_state_dict(torch.load(checkpoint_graph, map_location=torch.device('cpu')))
model.eval()

GCN(
  (conv1): GraphConv(in=3, out=16, normalization=both, activation=None)
  (dropout): Dropout(p=0.5, inplace=False)
  (conv2): GraphConv(in=16, out=1, normalization=both, activation=None)
)

In [13]:
from func.graph_learning import VoxelGraphDataset
dataset = VoxelGraphDataset(set_test, with_ground_truth_labels=True)

In [15]:
for voxel_graph in dataset:
    model_output = torch.sigmoid(model(voxel_graph, voxel_graph.ndata['feat']))
    predictions = (model_output > 0.5).type(torch.FloatTensor)

    labels = voxel_graph.ndata['label']

    accuracy = (predictions == labels).float().mean()
    print(accuracy)
    print("done")

tensor(0.5376)
done
tensor(0.5252)
done
tensor(0.5643)
done
tensor(0.5717)
done


In [11]:
print(set_test[0])

DiGraph with 54438 nodes and 1172568 edges
